### Notebook to parse scraped html to produce cleaned text of TCC decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). NOTE: Users must also comply with upstream [licensing](https://www.tcc-cci.gc.ca/en/pages/important-notices) for the TCC source data, as well as requests on source urls not to allow indexing of the documents by search engines to protect privacy. As a result, users must not make the data available in formats or locations that can be indexed by search engines.
`
Dataset & Code to be cited as: 

    Sean Rehaag, "Tax Court of Canada Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/tcc/>.

Notes:

(1) Data Source: [Tax Court of Canada](https://www.tcc-cci.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Tax Court of Canada website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Tax Court of Canada.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Tax Court of Canada website's [Terms of Use](https://www.tcc-cci.gc.ca/en/pages/important-notices).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 

(6) Limitation: Only includes cases with neutral citation, which began to be used in 2003

(7) Delay: Decisions may take many months to be translated (sometimes over a year). As a result, in the most recent years, decisions may only be available in one language.


In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/TCC/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/tcc_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/tcc_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/tcc_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')

# set years sought
start_year = 2003 
end_year = 2023

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    try:
        with open(file) as f:
            data = json.load(f)
            results.append(data)
    except Exception as e:
        print('ERROR READING FILE: ',file)
        continue

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)

# NOTE: ERRORS IN PRE-NEUTRAL CITATION FILE READING CAN BE IGNORED
# These errors are likely due to file name length limits, because the file
# names are long where there are a large number of dockets in a single case
# where neutral citation is not used.


20998


100%|██████████| 20998/20998 [00:16<00:00, 1290.73it/s]


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = citation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Run this in parallel to speed up

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

# Convert the pandas DataFrame to a Dask DataFrame
ddf = dd.from_pandas(df, npartitions=16) 

# Apply the function in parallel
ddf['text'] = ddf.html.apply(get_text)

# Convert the Dask DataFrame back to a pandas DataFrame
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        df = ddf.compute()



[                                        ] | 0% Completed | 220.40 us

d:\AI-Projects\venv311\Lib\site-packages\dask\dataframe\core.py:4378: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('html', 'object'))

  warnings.warn(meta_warning(meta))


[########################################] | 100% Completed | 105.78 s


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 15050/15050 [00:23<00:00, 648.21it/s]


In [7]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# rename decision_date to document_date
df = df.rename(columns={'decision_date': 'document_date'})

# add dataset column
df['dataset'] = 'TCC'

# add other column
df['other'] = ''

# reorder columns
list_cols = ['citation',
             'citation2', 
             'dataset', 
             'year', 
             'name', 
             'language', 
             'document_date', 
             'source_url', 
             'scraped_timestamp', 
             'unofficial_text',
             'other']

df = df[list_cols]

# reset index
df = df.reset_index(drop=True)

In [8]:
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2003 TCC 1,,TCC,2003,Rodier c. La Reine,en,2003-01-30,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2022-09-02,Rodier c. La Reine\nCourt (s) Database\nTax Co...,
1,2003 TCC 100,,TCC,2003,Doering v. M.N.R.,en,2003-03-12,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2022-09-02,Doering v. M.N.R.\nCourt (s) Database\nTax Cou...,
2,2003 TCC 101,,TCC,2003,Loewen v. The Queen,en,2003-03-14,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2022-09-02,Loewen v. The Queen\nCourt (s) Database\nTax C...,
3,2003 TCC 102,,TCC,2003,Dixon v. The Queen,en,2003-03-12,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2022-09-02,Dixon v. The Queen\nCourt (s) Database\nTax Co...,
4,2003 TCC 103,,TCC,2003,S K Management Inc. v. The Queen,en,2003-03-07,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2022-09-02,S K Management Inc. v. The Queen\nCourt (s) Da...,


In [9]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
15045,2023 CCI 73,,TCC,2023,Sanchez c. Le Roi,fr,2023-05-25,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2023-12-01,Sanchez c. Le Roi\nBase de données – Cour (s)\...,
15046,2023 CCI 79,,TCC,2023,SPE Valeur Assurable Inc. c. Le Roi,fr,2023-06-01,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2023-12-01,SPE Valeur Assurable Inc. c. Le Roi\nBase de d...,
15047,2023 CCI 9,,TCC,2023,Payette c. M.R.N.,fr,2023-01-20,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2023-12-01,Payette c. M.R.N.\nBase de données – Cour (s)\...,
15048,2023 CCI 97,,TCC,2023,Boisselle c. Le Roi,fr,2023-07-10,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2023-12-01,Boisselle c. Le Roi\nBase de données – Cour (s...,
15049,2023 TCC 26,,TCC,2023,Doussot c. Le Roi,fr,2023-05-25,https://decision.tcc-cci.gc.ca/tcc-cci/decisio...,2023-12-01,Doussot c. Le Roi\nBase de données – Cour (s)\...,


### Export data

In [10]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [11]:
# export cleaned df to parquet
df.to_parquet(out_path_parquet)

In [12]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 21/21 [00:00<00:00, 24.47it/s]
